In [1]:
import dash
import dash_core_components as dcc 
import dash_html_components as html 
import dash_table 
from dash_table.Format import Format, Scheme, Sign, Symbol
import dash_table.FormatTemplate as FormatTemplate
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go 
import plotly.figure_factory as ffimport 
import pandas as pd
import plotly.graph_objs as go
import numpy as np
import itertools
import json
import datetime as dt 

import sys
sys.path.append('..')

from app import app

ENABLE_WEBSOCKET_SUPPORT = False
refresh_rate = 5 if ENABLE_WEBSOCKET_SUPPORT else 7
if ENABLE_WEBSOCKET_SUPPORT:
    import diginex.ccxt.websocket_support as ccxt
else:
    import ccxt
    
# Define my world
deriv_exchanges =['deribit','bitmex']

exchanges =  deriv_exchanges 

api_keys = {'deribit':'4v58Wk2hhiG9B','bitmex':'DM55KFt84AfdjJiyNjDBk_km'}
api_secrets = {'deribit':'QLXLOZCOHAEQ6XV247KEXAKPX43GZLT4','bitmex':'jPDWiZcKuXJtVpajawENQiBzCKO2U885i3TWU9WIihBBUZgc'}

exch_dict={}
for x in exchanges:
    exec('exch_dict[x]=ccxt.{}({{"apiKey": "{}", "secret": "{}"}})'.format(x, api_keys[x], api_secrets[x]))
for x,xccxt in exch_dict.items():
    xccxt.load_markets()

deribit = exch_dict['deribit']
bitmex = exch_dict['bitmex']

Xpto = ['BTC','ETH']

In [2]:
def get_instruments():
    deribit_all_ins={}
    for base in Xpto:
        base_list=[]
        for ins in deribit.markets:
            market = deribit.markets[ins]
            if market['type']=='future' and market['base']==base:
                base_list.append(ins)
        deribit_all_ins[base]=base_list
    bitmex_all_ins={}
    for base in Xpto:
        base_list=[]
        for ins in bitmex.markets:
            market = bitmex.markets[ins]
            if market['type'] in ('future','swap') and market['base']==base and market['info']['hasLiquidity']:
                expiry = market['info']['expiry']
                if expiry is None:
                    base_list.append(ins)
                else:
                    dt_expiry = dt.datetime.strptime(expiry,"%Y-%m-%dT%H:%M:%S.%fZ")
                    if dt_expiry > dt.datetime.now():
                        base_list.append(ins)
        bitmex_all_ins[base]=base_list
    return deribit_all_ins,bitmex_all_ins

In [3]:
deribit_all_ins , bitmex_all_ins = get_instruments()
deribit_ins = deribit_all_ins['BTC']
bitmex_ins= bitmex_all_ins['BTC']
instruments = deribit_ins + bitmex_ins


In [4]:
def get_exchanges_for_pair(pair):
    '''input: a pair
    output: a dictionary of ccxt exchange objects of the exchanges listing the pair
    '''
    return {x:exch_dict[x] for x in exch_dict if pair in list(exch_dict[x].load_markets().keys())}

In [5]:
def get_pairs_for_exchange(ex):
    '''input: an exchange
    output: a list of pairs '''
    d={}
    exec('d[ex]=ccxt.{}()'.format(ex))
    d[ex].load_markets()
    return d[ex].symbols

In [82]:
def get_order_books(pair,ex):
    '''pair is the pair string ,'BTC/USD'...
        returns a dictionary of order books for the pair
        special case for binance which API fails if # of parmaeters > 2
    '''
    order_books = {key: value.fetch_order_book(pair,limit=2000 ,
                        params={'full':1,'level':3,'limit_bids':0,'limit_asks':0,'type':'both'})
                        for key,value in ex.items() }
    if 'deribit' in order_books and 'BTC' in pair:
        bids_df = pd.DataFrame(order_books['deribit']['bids'])
        asks_df = pd.DataFrame(order_books['deribit']['asks'])
        bids_df[1]=bids_df[1]*10
        asks_df[1]= asks_df[1]*10
        order_books['deribit']['bids']=bids_df.values.tolist()
        order_books['deribit']['asks']=asks_df.values.tolist()
    return order_books

In [83]:
def aggregate_order_books(dict_of_order_books):
    '''dict_of_order_books is a dict of ccxt like order_books
        retuns a ccxt like dictionary order book sorted by prices 
    '''
    agg_dict_order_book = {}
    bids = []
    for x in dict_of_order_books:
        for bid in dict_of_order_books[x]['bids']:
            bids.append(bid+[x])
    asks = []
    for x in dict_of_order_books:
        for ask in dict_of_order_books[x]['asks']:
            asks.append(ask+[x])
    agg_dict_order_book['bids'] = (pd.DataFrame(bids)).sort_values(by=0,ascending=False).values.tolist()
    agg_dict_order_book['asks'] = (pd.DataFrame(asks)).sort_values(by=0,ascending=True).values.tolist()
    return agg_dict_order_book

In [84]:
def normalize_order_book(order_book,cutoff=.1,step=.001):
    '''order_book is a dictionary with keys bids asks timestamp datetime ...
    where bids is a list of list [[bid,bid_size]] and 
    asks is a list of list [[ask,ask_size]]
    this is returned by ccxt.'exchange'.fetch_order_book()
    returns a dataframe with columns [ask, ask_size, ask_size_$, cum_ask_size_$, bid_, bid_size, bid_size_$, cum_bid_size_$]
    and an index of shape np.linspace(1 - cutoff,1 + cutoff ,step =.001 ~ 10 bps)  
    '''
    try:
        rounding = int(np.ceil(-np.log(step)/np.log(10)))
        agg = True
    except:
        agg = False
    bid_side = pd.DataFrame(order_book['bids'],columns=['bid','bid_size_$','exc'])
    bid_side['cum_bid_size_$'] = bid_side['bid_size_$'].cumsum()
    ask_side = pd.DataFrame(order_book['asks'],columns=['ask','ask_size_$','exc'])
    ask_side['cum_ask_size_$'] = ask_side['ask_size_$'].cumsum()
    ref = (bid_side['bid'][0]+ask_side['ask'][0])/2
    bid_side['bid%'] = round(bid_side['bid']/ref,rounding) if agg else bid_side['bid']/ref
    ask_side['ask%'] = round(ask_side['ask']/ref,rounding) if agg else ask_side['ask']/ref
    bid_side = bid_side[bid_side['bid%']>=1-cutoff]
    ask_side = ask_side[ask_side['ask%']<=1+cutoff]
    bid_side['bid_size'] = bid_side['bid_size_$']/bid_side['bid']
    bid_side['cum_bid_size'] = bid_side['bid_size'].cumsum()
    ask_side['ask_size'] = ask_side['ask_size_$']/ask_side['ask']
    ask_side['cum_ask_size'] = ask_side['ask_size'].cumsum()
    normalized_bids = pd.DataFrame(bid_side.groupby('bid%',sort=False).mean()['bid'])
    normalized_bids.columns = ['bid']
    normalized_bids['bid_size'] = bid_side.groupby('bid%',sort=False).sum()['bid_size']
    normalized_bids['cum_bid_size'] = normalized_bids['bid_size'].cumsum()
    normalized_bids['bid_size_$'] = bid_side.groupby('bid%',sort=False).sum()['bid_size_$']
    normalized_bids['cum_bid_size_$'] = normalized_bids['bid_size_$'].cumsum()
    normalized_bids['average_bid_fill'] = normalized_bids['cum_bid_size_$']/normalized_bids['cum_bid_size']
    normalized_bids['bids_exc']=bid_side.groupby('bid%',sort=False).apply(lambda x: x['exc'].loc[x['bid_size'].idxmax()])
    normalized_asks = pd.DataFrame(ask_side.groupby('ask%',sort=False).mean()['ask'])
    normalized_asks.columns = ['ask']
    normalized_asks['ask_size'] = ask_side.groupby('ask%',sort=False).sum()['ask_size']
    normalized_asks['cum_ask_size'] = normalized_asks['ask_size'].cumsum()
    normalized_asks['ask_size_$'] = ask_side.groupby('ask%',sort=False).sum()['ask_size_$']
    normalized_asks['cum_ask_size_$'] = normalized_asks['ask_size_$'].cumsum()
    normalized_asks['average_ask_fill']=normalized_asks['cum_ask_size_$']/normalized_asks['cum_ask_size']
    normalized_asks['asks_exc']=ask_side.groupby('ask%',sort=False).apply(lambda x: x['exc'].loc[x['ask_size'].idxmax()])
    book=pd.concat([normalized_asks,normalized_bids],sort=False)
    return book

In [85]:
def build_book(order_books,pair,exchanges,cutoff=.1,step=0.001):
    ''' gets order books aggreagtes them then normalizes
        returns a dataframe
    '''
    return normalize_order_book(aggregate_order_books({key:order_books[key] for key in exchanges}),cutoff,step)

In [86]:
pair = 'BTC-PERPETUAL'

In [87]:
ex = get_exchanges_for_pair(pair)

In [88]:
ex

{'deribit': <ccxt.deribit.deribit at 0x7f4988f77b38>}

In [89]:
obs = get_order_books(pair,ex)

In [90]:
obs['deribit']

{'bids': [[5547.25, 271220.0],
  [5547.0, 231400.0],
  [5546.75, 270.0],
  [5546.25, 7500.0],
  [5546.0, 3740.0],
  [5545.75, 40220.0],
  [5545.5, 2000.0],
  [5545.25, 1760.0],
  [5545.0, 10000.0],
  [5544.5, 45970.0],
  [5544.25, 4930.0],
  [5544.0, 15000.0],
  [5543.75, 7990.0],
  [5543.5, 65650.0],
  [5543.25, 55440.0],
  [5543.0, 33400.0],
  [5542.5, 16670.0],
  [5542.0, 2000.0],
  [5541.75, 37310.0],
  [5541.5, 1710.0]],
 'asks': [[5547.5, 61550.0],
  [5547.75, 13790.0],
  [5548.0, 92690.0],
  [5548.25, 70050.0],
  [5548.5, 92000.0],
  [5548.75, 6000.0],
  [5549.0, 6510.0],
  [5549.25, 76000.0],
  [5549.5, 1680.0],
  [5549.75, 33320.0],
  [5550.0, 270.0],
  [5550.25, 48130.0],
  [5550.5, 200.0],
  [5550.75, 30530.0],
  [5551.0, 46760.0],
  [5551.25, 7510.0],
  [5551.5, 30000.0],
  [5551.75, 1610.0],
  [5552.0, 50700.0],
  [5552.5, 64950.0]],
 'timestamp': 1555998220000,
 'datetime': '2019-04-23T05:43:40.000Z',
 'nonce': None}

In [91]:
exch_dict

{'deribit': <ccxt.deribit.deribit at 0x7f4988f77b38>,
 'bitmex': <ccxt.bitmex.bitmex at 0x7f49b43b6908>}

In [92]:
exch_dict['bitmex'].markets

{'.EVOL7D': {'percentage': True,
  'precision': {'amount': None, 'price': 2},
  'limits': {'amount': {'min': None, 'max': None},
   'price': {'min': 0.01, 'max': None},
   'cost': {'min': None, 'max': None}},
  'id': '.EVOL7D',
  'symbol': '.EVOL7D',
  'base': 'ETH',
  'quote': 'XXX',
  'baseId': 'ETH',
  'quoteId': 'XXX',
  'active': False,
  'taker': None,
  'maker': None,
  'type': 'future',
  'spot': False,
  'swap': False,
  'future': True,
  'prediction': False,
  'info': {'symbol': '.EVOL7D',
   'rootSymbol': 'EVOL',
   'state': 'Unlisted',
   'typ': 'MRIXXX',
   'listing': None,
   'front': None,
   'expiry': None,
   'settle': None,
   'relistInterval': None,
   'inverseLeg': '',
   'sellLeg': '',
   'buyLeg': '',
   'optionStrikePcnt': None,
   'optionStrikeRound': None,
   'optionStrikePrice': None,
   'optionMultiplier': None,
   'positionCurrency': '',
   'underlying': 'ETH',
   'quoteCurrency': 'XXX',
   'underlyingSymbol': '.EVOL7D',
   'reference': 'BMEX',
   'reference

In [93]:
def get_instruments():
    deribit_all_ins={}
    for base in Xpto:
        base_list=[]
        for ins in deribit.markets:
            market = deribit.markets[ins]
            if market['type']=='future' and market['base']==base:
                base_list.append(ins)
        deribit_all_ins[base]=base_list
    bitmex_all_ins={}
    for base in Xpto:
        base_list=[]
        for ins in bitmex.markets:
            if not ins.startswith('.') and '_' not in ins:
                market = bitmex.markets[ins]
                if market['type'] in ('future','swap') and market['base']==base:
                    expiry = market['info']['expiry']
                    if expiry is None:
                        base_list.append(ins)
                    else:
                        dt_expiry = dt.datetime.strptime(expiry,"%Y-%m-%dT%H:%M:%S.%fZ")
                        if dt_expiry > dt.datetime.now():
                            base_list.append(ins)
            bitmex_all_ins[base]=base_list
    return deribit_all_ins,bitmex_all_ins

In [94]:
get_instruments()

({'BTC': ['BTC-27SEP19', 'BTC-28JUN19', 'BTC-PERPETUAL'],
  'ETH': ['ETH-28JUN19', 'ETH-27SEP19', 'ETH-PERPETUAL']},
 {'BTC': ['BTC/USD', 'XBTM19', 'XBTU19'], 'ETH': ['ETH/USD', 'ETHM19']})

In [95]:
for ins in bitmex.markets:
    if not ins.startswith('.'):
        print (ins)

XRPM19
BCHM19
ADAM19
EOSM19
TRXM19
BTC/USD
XBT7D_U105
XBT7D_D95
XBTM19
XBTU19
ETH/USD
ETHM19
LTCM19


In [96]:
bitmex.markets['XBT7D_U105']

{'percentage': True,
 'precision': {'amount': 0, 'price': 5},
 'limits': {'amount': {'min': 1.0, 'max': 10000000.0},
  'price': {'min': 1e-05, 'max': 0.1},
  'cost': {'min': None, 'max': None}},
 'id': 'XBT7D_U105',
 'symbol': 'XBT7D_U105',
 'base': 'BTC',
 'quote': 'BTC',
 'baseId': 'XBT',
 'quoteId': 'XBT',
 'active': True,
 'taker': 0,
 'maker': 0,
 'type': 'future',
 'spot': False,
 'swap': False,
 'future': True,
 'prediction': False,
 'info': {'symbol': 'XBT7D_U105',
  'rootSymbol': 'XBT',
  'state': 'Open',
  'typ': 'OCECCS',
  'listing': '2019-04-19T12:00:00.000Z',
  'front': '2019-04-19T12:00:00.000Z',
  'expiry': '2019-04-26T12:00:00.000Z',
  'settle': '2019-04-26T12:00:00.000Z',
  'relistInterval': '2000-01-08T00:00:00.000Z',
  'inverseLeg': '',
  'sellLeg': '',
  'buyLeg': '',
  'optionStrikePcnt': 1.05,
  'optionStrikeRound': 250,
  'optionStrikePrice': 5500,
  'optionMultiplier': -0.1,
  'positionCurrency': '',
  'underlying': 'XBT',
  'quoteCurrency': 'XBT',
  'underlyin

In [97]:
expiry=deribit.markets['BTC-28JUN19']['info']

In [98]:
deribit.fetch_balance()

{'BTC': {'free': 0.25623834, 'used': 0.015057129, 'total': 0.276238348},
 'free': {'BTC': 0.25623834},
 'used': {'BTC': 0.015057129},
 'total': {'BTC': 0.276238348}}

In [99]:
pair = 'BTC/USD'

In [100]:
bitmex_balance=bitmex.fetch_balance()
bitmex_balance
b_balance={key:bitmex_balance[key] for key in ['free','used','total']}
b_balance['exc']='bitmex'

In [101]:
deribit_balance=deribit.fetch_balance()

In [102]:
d_balance={key:deribit_balance[key] for key in ['free','used','total']}
d_balance['exc']='deribit'
d_balance

{'free': {'BTC': 0.25623834},
 'used': {'BTC': 0.015057129},
 'total': {'BTC': 0.276238348},
 'exc': 'deribit'}

In [103]:
d_df=pd.DataFrame(d_balance)
d_df

,free,used,total,exc
BTC,0.256238,0.015057,0.276238,deribit


In [104]:
b_df=pd.DataFrame(b_balance)
b_df

,free,used,total,exc
BTC,0.211717,0.0,0.211717,bitmex


In [105]:
b=pd.concat([d_df,b_df])

In [106]:
b[['exc','used','free','total']]

,exc,used,free,total
BTC,deribit,0.015057,0.256238,0.276238
BTC,bitmex,0.000000,0.211717,0.211717


In [107]:
def get_balances():
    deribit_balance=deribit.fetch_balance()
    bitmex_balance=bitmex.fetch_balance()
    d_balance={key:deribit_balance[key] for key in ['free','used','total']}
    d_balance['exc']='deribit'
    b_balance={key:bitmex_balance[key] for key in ['free','used','total']}
    b_balance['exc']='bitmex'
    b=pd.concat([pd.DataFrame(b_balance),pd.DataFrame(d_balance)])
    b = b[['exc','used','free','total']]
    return b

In [108]:
get_balances()

,exc,used,free,total
BTC,bitmex,0.000000,0.211717,0.211717
BTC,deribit,0.015057,0.256238,0.276238


In [109]:
250000*179

44750000

In [110]:
pd.DataFrame((deribit.private_get_positions())['result'])

,amount,averagePrice,averageUsdPrice,currency,delta,direction,estLiqPrice,floatingPl,floatingUsdPl,indexPrice,...,instrument,kind,maintenanceMargin,markPrice,openOrderMargin,profitLoss,realizedPl,settlementPrice,size,sizeEth
0,-0.2,0.000500,1.78949,NaN,-0.000510,sell,NaN,-0.000016,0.040924,5545.39,...,BTC-28JUN19-20000-C,option,0.015057,0.000286,0.0,0.000043,0.0,0.000207,-0.2,NaN
1,-4411.0,182.885478,NaN,eth,-24.832517,sell,-230.31,-0.589189,NaN,174.51,...,ETH-28JUN19,future,0.248942,177.630000,0.0,0.713598,0.0,0.000000,-4411.0,-24.832517


In [111]:
(bitmex.private_get_position()[0])

{'account': 787492,
 'symbol': 'ETHM19',
 'currency': 'XBt',
 'underlying': 'ETH',
 'quoteCurrency': 'XBT',
 'commission': 0.0025,
 'initMarginReq': 0.3333333333333333,
 'maintMarginReq': 0.01,
 'riskLimit': 5000000000,
 'leverage': 3,
 'crossMargin': False,
 'deleveragePercentile': None,
 'rebalancedPnl': -1164508,
 'prevRealisedPnl': 1171595,
 'prevUnrealisedPnl': 0,
 'prevClosePrice': 0.03308,
 'openingTimestamp': '2019-04-23T05:00:00.000Z',
 'openingQty': 0,
 'openingCost': -1158000,
 'openingComm': -6508,
 'openOrderBuyQty': 0,
 'openOrderBuyCost': 0,
 'openOrderBuyPremium': 0,
 'openOrderSellQty': 0,
 'openOrderSellCost': 0,
 'openOrderSellPremium': 0,
 'execBuyQty': 0,
 'execBuyCost': 0,
 'execSellQty': 0,
 'execSellCost': 0,
 'execQty': 0,
 'execCost': 0,
 'execComm': 0,
 'currentTimestamp': '2019-04-23T05:00:02.082Z',
 'currentQty': 0,
 'currentCost': -1158000,
 'currentComm': -6508,
 'realisedCost': -1158000,
 'unrealisedCost': 0,
 'grossOpenCost': 0,
 'grossOpenPremium': 0,


In [112]:
[ins for ins in deribit.fetch_markets() if ins['future'] is True][1]['info']

{'kind': 'future',
 'baseCurrency': 'BTC',
 'currency': 'USD',
 'minTradeSize': 1,
 'instrumentName': 'BTC-28JUN19',
 'isActive': True,
 'settlement': 'month',
 'created': '2018-12-21 14:06:11 GMT',
 'tickSize': 0.25,
 'pricePrecision': 1,
 'expiration': '2019-06-28 08:00:00 GMT',
 'contractSize': 10.0}

In [113]:
deribit.markets['BTC-PERPETUAL']

{'percentage': True,
 'precision': {'amount': 1, 'price': 0.25},
 'limits': {'amount': {'min': 1}, 'price': {'min': 0.25}},
 'id': 'BTC-PERPETUAL',
 'symbol': 'BTC-PERPETUAL',
 'base': 'BTC',
 'quote': 'USD',
 'active': True,
 'type': 'future',
 'spot': False,
 'future': True,
 'option': False,
 'info': {'kind': 'future',
  'baseCurrency': 'BTC',
  'currency': 'USD',
  'minTradeSize': 1,
  'instrumentName': 'BTC-PERPETUAL',
  'isActive': True,
  'settlement': 'perpetual',
  'created': '2018-08-14 10:24:47 GMT',
  'tickSize': 0.25,
  'pricePrecision': 1,
  'expiration': '3000-01-01 08:00:00 GMT',
  'contractSize': 10.0}}

In [114]:
deribit.fetch_ohlcv("BTC-PERPETUAL",'1d')

[[1555977600000, 5547.75, 5547.75, 5547.5, 5547.5, 5081.0]]

In [115]:
deribit.fetch_ticker('BTC-PERPETUAL')['last']

5547.5

In [40]:
bitmex.fetch_ticker('BTC/USD')['last']

5512.0

In [41]:
def get_d1_instruments():
    all_ins={}
    for exc,exc_obj in exch_dict.items():
        all_ins[exc]={}
        for base in Xpto:
            base_list=[]
            for ins in getattr(exc_obj,'markets'):
                market = getattr(exc_obj,'markets')[ins]
                if market['type'] in ('future','swap') and market['base']==base and not ins.startswith('.') and '_' not in ins:
                    if exc == 'bitmex':
                        expiry = market['info']['expiry']
                        if expiry is None:
                            base_list.append(ins)
                        else:
                            dt_expiry = dt.datetime.strptime(expiry,"%Y-%m-%dT%H:%M:%S.%fZ")
                            if dt_expiry > dt.datetime.now():
                                base_list.append(ins)
                    else:
                        base_list.append(ins)
            all_ins[exc][base]=base_list
    
    return all_ins

In [42]:
(3587/3450-1)

0.03971014492753633

In [43]:
(188.38/180.08)-1

0.04609062638827188

In [44]:
get_d1_instruments()

{'deribit': {'BTC': ['BTC-27SEP19', 'BTC-28JUN19', 'BTC-PERPETUAL'],
  'ETH': ['ETH-28JUN19', 'ETH-27SEP19', 'ETH-PERPETUAL']},
 'bitmex': {'BTC': ['BTC/USD', 'XBTM19', 'XBTU19'],
  'ETH': ['ETH/USD', 'ETHM19']}}

In [45]:
deribit.fetch_balance()

{'BTC': {'free': 0.2562404, 'used': 0.015055075, 'total': 0.276240402},
 'free': {'BTC': 0.2562404},
 'used': {'BTC': 0.015055075},
 'total': {'BTC': 0.276240402}}

In [53]:
deribit_open_orders = []
for coin in deribit_all_ins:
    for symbol in deribit_all_ins[coin]:
        orders = deribit.fetch_open_orders(symbol)
        deribit_open_orders+= orders

In [54]:
deribit_open_orders

[{'info': {'orderId': 'ETH-449960704',
   'type': 'limit',
   'instrument': 'ETH-28JUN19',
   'direction': 'sell',
   'price': 177.0,
   'label': '',
   'amount': 10.0,
   'quantity': 10,
   'filledQuantity': 0,
   'filledAmount': 0.0,
   'avgPrice': 0.0,
   'commission': 0.0,
   'created': 1555997079439,
   'lastUpdate': 1555997079439,
   'state': 'open',
   'postOnly': False,
   'reduceOnly': False,
   'api': True,
   'max_show': 10,
   'maxShow': 10,
   'maxShowAmount': 10,
   'adv': False},
  'id': 'ETH-449960704',
  'timestamp': 1555997079439,
  'datetime': '2019-04-23T05:24:39.439Z',
  'lastTradeTimestamp': None,
  'symbol': 'ETH-28JUN19',
  'type': 'limit',
  'side': 'sell',
  'price': 177.0,
  'amount': 10.0,
  'cost': 0.0,
  'average': 0.0,
  'filled': 0.0,
  'remaining': 10.0,
  'status': 'open',
  'fee': {'cost': 0.0, 'currency': 'BTC'},
  'trades': None}]

In [48]:
deribit_all_ins

{'BTC': ['BTC-27SEP19', 'BTC-28JUN19', 'BTC-PERPETUAL'],
 'ETH': ['ETH-28JUN19', 'ETH-27SEP19', 'ETH-PERPETUAL']}

In [52]:
deribit.fetch_closed_orders('ETH-28JUN19')

[]

In [58]:
deribit.fetch_open_orders('ETH-28JUN19')

[{'info': {'orderId': 'ETH-449960704',
   'type': 'limit',
   'instrument': 'ETH-28JUN19',
   'direction': 'sell',
   'price': 177.0,
   'label': '',
   'amount': 10.0,
   'quantity': 10,
   'filledQuantity': 0,
   'filledAmount': 0.0,
   'avgPrice': 0.0,
   'commission': 0.0,
   'created': 1555997079439,
   'lastUpdate': 1555997079439,
   'state': 'open',
   'postOnly': False,
   'reduceOnly': False,
   'api': True,
   'max_show': 10,
   'maxShow': 10,
   'maxShowAmount': 10,
   'adv': False},
  'id': 'ETH-449960704',
  'timestamp': 1555997079439,
  'datetime': '2019-04-23T05:24:39.439Z',
  'lastTradeTimestamp': None,
  'symbol': 'ETH-28JUN19',
  'type': 'limit',
  'side': 'sell',
  'price': 177.0,
  'amount': 10.0,
  'cost': 0.0,
  'average': 0.0,
  'filled': 0.0,
  'remaining': 10.0,
  'status': 'open',
  'fee': {'cost': 0.0, 'currency': 'BTC'},
  'trades': None}]

In [81]:
bitmex.private_get_position()

[{'account': 787492,
  'symbol': 'ETHM19',
  'currency': 'XBt',
  'underlying': 'ETH',
  'quoteCurrency': 'XBT',
  'commission': 0.0025,
  'initMarginReq': 0.3333333333333333,
  'maintMarginReq': 0.01,
  'riskLimit': 5000000000,
  'leverage': 3,
  'crossMargin': False,
  'deleveragePercentile': None,
  'rebalancedPnl': -1164508,
  'prevRealisedPnl': 1171595,
  'prevUnrealisedPnl': 0,
  'prevClosePrice': 0.03308,
  'openingTimestamp': '2019-04-23T05:00:00.000Z',
  'openingQty': 0,
  'openingCost': -1158000,
  'openingComm': -6508,
  'openOrderBuyQty': 0,
  'openOrderBuyCost': 0,
  'openOrderBuyPremium': 0,
  'openOrderSellQty': 0,
  'openOrderSellCost': 0,
  'openOrderSellPremium': 0,
  'execBuyQty': 0,
  'execBuyCost': 0,
  'execSellQty': 0,
  'execSellCost': 0,
  'execQty': 0,
  'execCost': 0,
  'execComm': 0,
  'currentTimestamp': '2019-04-23T05:00:02.082Z',
  'currentQty': 0,
  'currentCost': -1158000,
  'currentComm': -6508,
  'realisedCost': -1158000,
  'unrealisedCost': 0,
  'gro